# Federated Protein Embeddings and Task Model Fitting with BioNeMo

This example notebook shows how to obtain protein learned representations in the form of embeddings using the ESM-1nv pre-trained model. The model is trained with NVIDIA's BioNeMo framework for Large Language Model training and inference. For more details, please visit NVIDIA BioNeMo Service at https://www.nvidia.com/en-us/gpu-cloud/bionemo.

This notebook will walk you through the task fitting workflow in the following sections:

* 
*
*

### Install requirements
Let's start by installing and importing library dependencies. We'll use requests to interact with the BioNeMo service, BioPython to parse FASTA sequences into SeqRecord objects, scikit-learn for classification tasks, and matplotlib for visualization.

In [30]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 3.1 MB 2.8 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


### Obtaining the protein embeddings using the BioNeMo ESM-1nv model
Using BioNeMo, users can obtain numerical vector representations of protein sequences called embeddings. Protein embeddings can then be used for visualization or making downstream predictions.

Here we are interested in training a neural network to predict subcellular location from an embedding.

The data we will be using comes from the paper [Light attention predicts protein location from the language of life](https://academic.oup.com/bioinformaticsadvances/article/1/1/vbab035/6432029) by Stärk et al. In this paper, the authors developed a machine learning algorithm to predict the subcellular location of proteins from sequence through protein langage models that are similar to those hosted by BioNeMo. Protein subcellular location refers to where the protein localizes in the cell, for example a protein my be expressed in the Nucleus or in the Cytoplasm. Knowing where proteins localize can provide insights into the underlying mechanisms of cellular processes and help identify potential targets for drug development. The following image includes a few examples of subcellular locations in an animal cell:


(Image freely available at https://pixabay.com/images/id-48542)

### Dataset sourcing
For our target input sequences, we will point to FASTA sequences in a benchmark dataset called Fitness Landscape Inference for Proteins (FLIP). FLIP encompasses experimental data across adeno-associated virus stability for gene therapy, protein domain B1 stability and immunoglobulin binding, and thermostability from multiple protein families.

In [31]:
# Example protein dataset location
fasta_url= "http://data.bioembeddings.com/public/FLIP/fasta/scl/mixed_soft.fasta"

First, we define the source of example protein dataset with the FASTA sequences. This data follows the [biotrainer](https://github.com/sacdallago/biotrainer/blob/main/docs/data_standardization.md) standard, so it includes information about the class in the FASTA header, and the protein sequence. Here are two example sequences in this file:

```
>Sequence1 TARGET=Cell_membrane SET=train VALIDATION=False
MMKTLSSGNCTLNVPAKNSYRMVVLGASRVGKSSIVSRFLNGRFEDQYTPTIEDFHRKVYNIHGDMYQLDILDTSGNHPFPAM
RRLSILTGDVFILVFSLDSRESFDEVKRLQKQILEVKSCLKNKTKEAAELPMVICGNKNDHSELCRQVPAMEAELLVSGDENC
AYFEVSAKKNTNVNEMFYVLFSMAKLPHEMSPALHHKISVQYGDAFHPRPFCMRRTKVAGAYGMVSPFARRPSVNSDLKYIKA
KVLREGQARERDKCSIQ
>Sequence4833 TARGET=Nucleus SET=train VALIDATION=False
MARTKQTARKSTGGKAPRKQLATKAARKSAPATGGVKKPHRFRPGTVALREIRKYQKSTELLIRKLPFQRLVREIAQDFKTDL
RFQSSAVAALQEAAEAYLVGLFEDTNLCAIHAKRVTIMPKDIQLARRIRGERA
Note the following attributes in the FASTA header:
```

* `TARGET` attribute holds the subcellular location classification for the sequence, for instance Cell_membrane and Nucleus. This dataset includes a total of ten subcellelular location classes -- more on that below.
* `SET` attribute defines whether the sequence should be used for training (train) or testing (test)
* `VALIDATION` attribute defines whether the sequence should be used for validation (all sequences where this is True are also in set=train)

### Downloading the protein sequences and subcellular location annotations
In this step we download the FASTA file defined above and parse the sequences into a list of BioPython SeqRecord objects.



In [32]:
import io
import requests
from Bio import SeqIO

# Download the FASTA file from FLIP: https://github.com/J-SNACKKB/FLIP/tree/main/splits/scl
fasta_content = requests.get(fasta_url, headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x86)'
}).content.decode('utf-8')
fasta_stream = io.StringIO(fasta_content)

# Obtain a list of SeqRecords/proteins which contain sequence and attributes
# from the FASTA header
proteins = list(SeqIO.parse(fasta_stream, "fasta"))
print(f"Downloaded {len(proteins)} sequences")

Downloaded 13949 sequences


### Data splitting

### Federated embedding extraction

In [22]:
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder="jobs/embeddings",
    workspace="/tmp/nvflare/bionemo/embeddings",
    n_clients=1,
    threads=1
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-07-27 16:06:32,382 - SimulatorRunner - INFO - Create the Simulator Server.
2023-07-27 16:06:32,387 - Cell - INFO - server: creating listener on tcp://0:47077
2023-07-27 16:06:32,406 - Cell - INFO - server: created backbone external listener for tcp://0:47077
2023-07-27 16:06:32,407 - ConnectorManager - INFO - 5655: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-07-27 16:06:32,410 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:49429] is starting
2023-07-27 16:06:32,912 - Cell - INFO - server: created backbone internal listener for tcp://localhost:49429
2023-07-27 16:06:32,915 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:47077] is starting
2023-07-27 16:06:33,006 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 48497
2023-07-27 16:06:33,007 - SimulatorRunner - INFO - Deploy the Apps.
2023-07-27 16:06:33,516 - SimulatorRunner - INFO - Create the simulate

[NeMo W 2023-07-27 16:06:49 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.


2023-07-27 16:06:50,294 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job, task_name=bionemo_inference, task_id=2ef06425-ab8c-4ed1-8f54-2997ef69cd97]: assigned task to client site-1: name=bionemo_inference, id=2ef06425-ab8c-4ed1-8f54-2997ef69cd97
2023-07-27 16:06:50,297 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job, task_name=bionemo_inference, task_id=2ef06425-ab8c-4ed1-8f54-2997ef69cd97]: sent task assignment to client. client_name:site-1 task_id:2ef06425-ab8c-4ed1-8f54-2997ef69cd97
2023-07-27 16:06:50,299 - GetTaskCommand - INFO - return task to client.  client_name: site-1  task_name: bionemo_inference   task_id: 2ef06425-ab8c-4ed1-8f54-2997ef69cd97  sharable_header_task_id: 2ef06425-ab8c-4ed1-8f54-2997ef69cd97


[NeMo W 2023-07-27 16:06:50 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[NeMo I 2023-07-27 16:06:50 utils:250] Restoring model from /tmp/nvflare/bionemo/embeddings/simulate_job/app_site-1/models/esm1nv.nemo
[NeMo I 2023-07-27 16:06:50 utils:254] Loading model class: bionemo.model.protein.esm1nv.esm1nv_model.ESM1nvModel
2023-07-27 16:06:50,421 - pytorch_lightning.utilities.rank_zero - INFO - GPU available: True (cuda), used: True
2023-07-27 16:06:50,421 - pytorch_lightning.utilities.rank_zero - INFO - TPU available: False, using: 0 TPU cores
2023-07-27 16:06:50,421 - pytorch_lightning.utilities.rank_zero - INFO - IPU available: False, using: 0 IPUs
2023-07-27 16:06:50,422 - pytorch_lightning.utilities.rank_zero - INFO - HPU available: False, using: 0 HPUs
[NeMo I 2023-07-27 16:06:50 exp_manager:374] Experiments will be logged at /tmp/nvflare/bionemo/embeddings/simulate_job/app_site-1/nemo_experiments/ESM1nv_Inference/2023-07-27_16-06-50
[NeMo I 2023-07-27 16:06:50 exp_manager:797] TensorboardLogger has been set up
[NeMo I 2023-07-27 16:06:50 utils:234] 
   

...

[NeMo I 2023-07-27 16:06:50 megatron_init:231] Rank 0 has data parallel group: [0]
[NeMo I 2023-07-27 16:06:50 megatron_init:234] All data parallel group ranks: [[0]]
[NeMo I 2023-07-27 16:06:50 megatron_init:235] Ranks 0 has data parallel rank: 0
[NeMo I 2023-07-27 16:06:50 megatron_init:243] Rank 0 has model parallel group: [0]
[NeMo I 2023-07-27 16:06:50 megatron_init:244] All model parallel group ranks: [[0]]
[NeMo I 2023-07-27 16:06:50 megatron_init:254] Rank 0 has tensor model parallel group: [0]
[NeMo I 2023-07-27 16:06:50 megatron_init:258] All tensor model parallel group ranks: [[0]]
[NeMo I 2023-07-27 16:06:50 megatron_init:259] Rank 0 has tensor model parallel rank: 0
[NeMo I 2023-07-27 16:06:50 megatron_init:273] Rank 0 has pipeline model parallel group: [0]
[NeMo I 2023-07-27 16:06:50 megatron_init:285] Rank 0 has embedding group: [0]
[NeMo I 2023-07-27 16:06:50 megatron_init:291] All pipeline model parallel group ranks: [[0]]
[NeMo I 2023-07-27 16:06:50 megatron_init:292]

[NeMo W 2023-07-27 16:06:50 modelPT:245] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
I0727 16:06:50.951505 140313256003392 distributed.py:244] Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/1
I0727 16:06:50.953786 140313256003392 distributed.py:248] ----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 1 processes
----------------------------------------------------------------------------------------------------

[NeMo W 2023-07-27 16:06:50 nemo_logging:349] /usr/local/lib/python3.8/dist-packages/apex/transformer/pipeline_parallel/utils.py:81: UserWarning: This function is only for unittest
      warnings.warn("This function is only for unittest")
    


[NeMo I 2023-07-27 16:06:51 text_memmap_dataset:349] Time building 0 / 1 mem-mapped files: 0:00:00.129021
[NeMo I 2023-07-27 16:06:51 text_memmap_dataset:114] Loading data files
[NeMo I 2023-07-27 16:06:51 text_memmap_dataset:205] Loading /tmp/data/FLIP/secondary_structure/test/x000.csv
[NeMo I 2023-07-27 16:06:51 text_memmap_dataset:117] Time loading 1 mem-mapped files: 0:00:00.003985
[NeMo I 2023-07-27 16:06:51 text_memmap_dataset:121] Computing global indices
[NeMo I 2023-07-27 16:06:51 mapped_dataset:206] Filtered out (ignored) 24 samples ( 340 / 364 )
2023-07-27 16:06:51,962 - pytorch_lightning.accelerators.cuda - INFO - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Predicting DataLoader 0:   0%|          | 0/3 [00:00<?, ?it/s]

I0727 16:06:51.962398 140313256003392 cuda.py:58] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting DataLoader 0: 100%|██████████| 3/3 [00:04<00:00,  1.38s/it]2023-07-27 16:06:57,277 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job]: got result from client site-1 for task: name=bionemo_inference, id=2ef06425-ab8c-4ed1-8f54-2997ef69cd97
2023-07-27 16:06:57,280 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference, peer=site-1, peer_run=simulate_job, peer_rc=OK, task_name=bionemo_inference, task_id=2ef06425-ab8c-4ed1-8f54-2997ef69cd97]: finished processing client result by bionemo_inference
2023-07-27 16:06:57,283 - SubmitUpdateCommand - INFO - submit_update process. client_name:site-1   task_id:2ef06425-ab8c-4ed1-8f54-2997ef69cd97
2023-07-27 16:06:57,465 - BioNeMoInference - INFO - [identity=simulator_server, run=simulate_job, wf=bionemo_inference]: task bionemo_inference exit with status TaskCompletionStatus.OK
2023-07-27 16:06:57,498 - BioNeMoInferenceProcess

### Inspecting the embeddings and labels
Embeddings returned from the BioNeMo server are vectors of fixed size for each input sequence. In other words, if we input 10 sequences, we will obtain a matrix `10xD`, where `D` is the size of the embedding (in the case of ESM-1nv, `D=768`). At a glance, these real-valued vector embeddings don't show any obvious features (see the printout in the next cell). But these vectors do contain information that can be used in downstream models to reveal properties of the protein, for example the subcellular location as we'll explore below.

In [34]:
# Let's also print all the labels

labels = set([meta['TARGET'] for meta in metadata])

for i, label in enumerate(labels):
    print(f"{i+1}. {label.replace('_', ' ')}")

NameError: name 'metadata' is not defined

### Training a MLP to predict subcellular location
To be able to classify proteins for their subcellular location, we train a simple scikit-learn Multi-layer Perceptron (MPL) classifier. The MLP model uses a network of hidden layers to fit the input embedding vectors to the model classes (the cellular locations above). In the call below, we define the MLP to use the Adam optimizer with a network of 32 hidden layers, defining a random state (or seed) for reproducibility, and trained for a maximum of 500 iterations.

In [54]:
from nvflare import SimulatorRunner    

simulator = SimulatorRunner(
    job_folder="jobs/fedavg",
    workspace="/tmp/nvflare/bionemo/fedavg",
    n_clients=1,
    threads=1
)
run_status = simulator.run()
print("Simulator finished with run_status", run_status)

2023-07-27 17:12:40,941 - SimulatorRunner - INFO - Create the Simulator Server.
2023-07-27 17:12:40,945 - Cell - INFO - server: creating listener on tcp://0:44967
2023-07-27 17:12:40,965 - Cell - INFO - server: created backbone external listener for tcp://0:44967
2023-07-27 17:12:40,967 - ConnectorManager - INFO - 12723: Try start_listener Listener resources: {'secure': False, 'host': 'localhost'}
2023-07-27 17:12:40,969 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:52586] is starting
2023-07-27 17:12:41,472 - Cell - INFO - server: created backbone internal listener for tcp://localhost:52586
2023-07-27 17:12:41,476 - nvflare.fuel.f3.sfm.conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:44967] is starting
2023-07-27 17:12:41,571 - nvflare.fuel.hci.server.hci - INFO - Starting Admin Server localhost on Port 52401
2023-07-27 17:12:41,572 - SimulatorRunner - INFO - Deploy the Apps.
2023-07-27 17:12:41,583 - SimulatorRunner - INFO - Create the simulat

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


############ MLPClassifier 2 [(768, 32), (32, 1)]
2023-07-27 17:12:44,743 - IntimeModelSelector - INFO - model selection weights control: None
2023-07-27 17:12:44,747 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job]: Server runner starting ...
2023-07-27 17:12:44,750 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job]: starting workflow scatter_gather_ctl (<class 'nvflare.app_common.workflows.scatter_and_gather.ScatterAndGather'>) ...
2023-07-27 17:12:44,752 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job]: Initializing ScatterAndGather workflow.
2023-07-27 17:12:44,763 - ServerRunner - INFO - [identity=simulator_server, run=simulate_job]: Workflow scatter_gather_ctl (<class 'nvflare.app_common.workflows.scatter_and_gather.ScatterAndGather'>) started
2023-07-27 17:12:44,765 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: Beginning ScatterAndGather training phase.
2023-07-27

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:598: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (8) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (8) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (8) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:679: ConvergenceWarning: Sto

2023-07-27 17:14:04,984 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: task train exit with status TaskCompletionStatus.OK
2023-07-27 17:14:05,114 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: Start aggregation.
2023-07-27 17:14:05,118 - DXOAggregator - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: aggregating 1 update(s) at round 34
2023-07-27 17:14:05,120 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: End aggregation.
2023-07-27 17:14:05,123 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: Start persist model on server.
2023-07-27 17:14:05,126 - ScatterAndGather - INFO - [identity=simulator_server, run=simulate_job, wf=scatter_gather_ctl]: End persist model on server.
2023-07-27 17:14:05,129 - ScatterAndGather - INFO - [identity=simulator_server, run=sim